In [71]:
from pyspark.sql import SparkSession

In [72]:
from pyspark.ml.regression import LinearRegression

In [73]:
spark = SparkSession.builder.appName('lreg').getOrCreate()

In [74]:
data = spark.read.csv('data/Ecommerce_Customers.csv', inferSchema = True, header = True)

In [75]:
data.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [76]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [77]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [78]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [79]:
# converting input columns into vector
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership'],
                            outputCol='features')     

In [80]:
# tranforming data
output = assembler.transform(data)

In [81]:
output.show(5)  # notice the extra column "features"

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [82]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [83]:
final_data = output.select('features', 'Yearly Amount Spent')

In [84]:
final_data.show(5)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
+--------------------+-------------------+
only showing top 5 rows



In [85]:
training_data, test_data = final_data.randomSplit([0.7,0.3])

In [86]:
training_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                353|
|   mean|  500.5959099994207|
| stddev|  81.62009386050957|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [87]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                147|
|   mean|  496.2358020374134|
| stddev|  73.66091868593546|
|    min|  314.4385182951061|
|    max|  725.5848140556806|
+-------+-------------------+



In [88]:
lr = LinearRegression(featuresCol='features', labelCol='Yearly Amount Spent')

In [89]:
lr_model = lr.fit(training_data)

In [90]:
test_result = lr_model.evaluate(test_data)

In [91]:
test_result.residuals.show()  # residuals are the difference between actual value is the value predicted

+--------------------+
|           residuals|
+--------------------+
|  -2.409816398353712|
|   7.702967142315401|
| -12.038750289123925|
|  12.370656317110388|
|    4.02570150235249|
|  0.8975349195070521|
|  -2.658600504738615|
|    19.0360840383496|
|-0.32496556590797354|
| -0.8017152579042772|
| -3.9524577199463806|
| -3.0335072510195005|
|  1.5300022604737933|
| -1.6753630472132954|
| -16.476782871879152|
|  -1.038072400496901|
|  12.249140235339667|
|   6.554376337360395|
| -17.032248042832407|
| -4.2535581560440505|
+--------------------+
only showing top 20 rows



In [92]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [93]:
# Compare to mean(average amount spend yearly), the rootMeanSquaredError is insignificant.
test_result.rootMeanSquaredError

10.062095036811462

In [94]:
# the r2 gives the 97% variance in the date
test_result.r2

0.9812125842160949

### To perform prediction on unlabeled data.

In [95]:
unlabeled_data = test_data.select('features')

In [96]:
unlabeled_data.show(5)

+--------------------+
|            features|
+--------------------+
|[30.8364326747734...|
|[30.9716756438877...|
|[31.0662181616375...|
|[31.1695067987115...|
|[31.3091926408918...|
+--------------------+
only showing top 5 rows



In [97]:
predictons = lr_model.transform(unlabeled_data)

In [98]:
predictons.show(5)

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8364326747734...| 469.9117168253433|
|[30.9716756438877...| 486.9356426145773|
|[31.0662181616375...| 460.9720434967983|
|[31.1695067987115...| 414.9858744851824|
|[31.3091926408918...|428.69501633758114|
+--------------------+------------------+
only showing top 5 rows

